# Microscope Setup

## This set of commands imports the necessary software, connects to the instruments, and should set the Z-axis to 5000 for a rough focus. This step takes about 20 seconds so be patient.

In [1]:
%load_ext autoreload
%autoreload 2
from napari.qt.threading import thread_worker
import control
import napari
import gui
import time
import numpy as np
import toml

s = """
[camera]
device = "zyla_camera"

[objective]
device = "ti_objective"
states = ["blank1", "4x", "blank2", "blank3", "10x", "blank4"]
zooms =  [0, 4, 0, 0, 10, 0]

[lightpath]
device = "ti_lightpath"
states = ["up", "left", "right", "down"]

[sola_light]
device = "sola_light"
port = "COM4"

[up_filter]
device = "ti_filter1"
states = ["mcherry", "egfp", "open1", "cy5", "brightfield", "open2"]

[down_filter]
device = "ti_filter2"

[stage]
device = "asi_stage"
port = "COM5"

[focus]
device = "ti_focus"

[right_filter]
device = "lambda_filter2"
port = "COM3"
states =  ["open", "435", "474", "536", "546", "572", "620", "630", "650", "780"]

[uv_filter]
device = "lambda_filter1"
port = "COM3"
states = ["uv", "open1", "?", "315nm", "??", "340nm", "???", "open2", "????", "brightfield"]

[uv]
device = "lambda_shutter1"
port = "COM3"

[ir]
device = "lambda_shutter2"
port = "COM3"
"""

conf = toml.loads(s)
c = control.load(conf, path="C:/Program Files/Micro-Manager-2.0")
c._core.setTimeoutMs(100000)
c.exposure = 100
c.objective = "4x"
c.z = 5000
c.binning = 2
c.lightpath = "right"
c._core.setProperty("camera", "TriggerMode", "Software (Recommended for Live Mode)")

# Live Imaging

In [6]:
c.exposure = 300

In [10]:
c.sola_light = 4

In [9]:
c.uv = "closed"
c.up_filter = "brightfield"
c.sola_light = 4
c.right_filter = "open"
c.exposure = 30
g = gui.live(c)

# Move the stage

In [17]:
c.xy = (0, 0)

# Pause the live imaging worker

In [11]:
g.pause()

In [12]:
c.x += 1000

## Resume the live imaging worker

In [13]:
g.resume()

## Tiled Acquisition

In [ ]:
c.uv = "open"
c.uv_filter = "uv"
c.up_filter = "open2"
c.sola_light = 0
c.right_filter = "620"
c.exposure = 100

In [ ]:
channels = ["cy5", "egfp", "435", "474", "536", "546", "572", "620", "630", "650", "780"]
def acq_func(xp, xs, ys):
    for j in range(5):
        for k, x in enumerate(xs):
            for l, y in enumerate(ys):
                c.xy = (x, y)
                for i, chan in enumerate(channels):
                    if chan == "cy5":
                        c.uv = "closed"
                        c.right_filter = "open"
                        c.up_filter = chan
                        c.exposure = 100
                        c.sola_light = 50
                    elif chan == "egfp":
                        c.uv = "closed"
                        c.right_filter = "open"
                        c.up_filter = chan
                        c.exposure = 500
                        c.sola_light = 100
                    else:
                        c.uv = "open"
                        c.up_filter = "open2"
                        c.right_filter = chan
                        c.sola_light = 0
                        c.exposure = 1000
    
                    c.wait()
                    xp.tile[i, j, k, l] = c.snap()
                    yield
                    
g, xp = gui.acquire(c,
                    file="test_file_9",
                    acq_func=acq_func,
                    overlap=0.1,
                    times=5,
                    channels=channels)

# Accessing image data mid-collection